In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
from Hackaton.python.read_pics import get_pics_from_file

In [3]:
pics_name = ["pics_0", "pics_1", "pics_2", "pics_3", "pics_4",
             "pics_5", "pics_6", "pics_7", "pics_8", "pics_9", "pics_NOKEY"]



## Transform to panda dataset


In [25]:
data = dict()
for i in range(17):
    data[i] = []
data["name"] = []

for pic_name in pics_name:
    pics_key, info = get_pics_from_file(f'Hackaton/data/{pic_name}.bin')
    for pics in pics_key:
        for i, pic in enumerate(pics):
            data[i].append(pic)
        data["name"].append(pic_name)

df = pd.DataFrame(data)
df.head(6)


Ouverture du fichier de pics Hackaton/data/pics_0.bin
Ouverture du fichier de pics Hackaton/data/pics_1.bin
Ouverture du fichier de pics Hackaton/data/pics_2.bin
Ouverture du fichier de pics Hackaton/data/pics_3.bin
Ouverture du fichier de pics Hackaton/data/pics_4.bin
Ouverture du fichier de pics Hackaton/data/pics_5.bin
Ouverture du fichier de pics Hackaton/data/pics_6.bin
Ouverture du fichier de pics Hackaton/data/pics_7.bin
Ouverture du fichier de pics Hackaton/data/pics_8.bin
Ouverture du fichier de pics Hackaton/data/pics_9.bin
Ouverture du fichier de pics Hackaton/data/pics_NOKEY.bin


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,name
0,0.968018,1.348877,0.230713,0.941162,0.811157,0.838623,0.671997,0.506592,0.765991,0.277100,0.477295,0.569458,0.371704,0.802002,0.634155,0.509033,0.567017,pics_0
1,0.754395,1.149902,0.137939,1.062012,0.589600,0.745239,0.734253,0.904541,0.669556,0.085449,0.650635,0.791626,0.339355,0.538330,0.848999,0.687256,0.356445,pics_0
2,0.765991,1.292725,0.225830,1.033325,0.750122,0.550537,0.760498,0.451660,0.767822,0.393066,0.463867,0.661011,0.431519,0.609741,0.845947,0.637207,0.558472,pics_0
3,0.573730,0.880127,1.459351,0.375977,1.184692,0.697021,0.614014,0.910645,1.012573,0.670776,0.264282,0.765991,0.631714,0.355835,0.656128,1.054688,0.786133,pics_0
4,0.428467,0.922241,1.361694,0.296631,1.135254,0.856934,0.629883,0.707397,0.797729,0.759888,0.233154,0.765991,0.765991,0.623169,0.587158,0.908813,0.946045,pics_0
5,0.908203,1.404419,0.296021,0.689697,0.859375,0.614014,0.723877,0.817871,0.854492,0.184326,0.659180,0.805054,0.373535,0.736084,0.798340,0.849609,0.279541,pics_0


In [47]:
y = (df.iloc[:, 17].to_numpy())
X = (df.iloc[:, :-1].to_numpy())

print("X shape = " + str(X.shape))
print("y shape = " + str(y.shape))

(97868,)
(97868, 17)
X shape = (97868, 17)
y shape = (97868,)


(17, 17)


In [ ]:
# List of name (X)


In [48]:
# Pre-processing
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition


X_size = 17     # Number of features (Pics after PCA)
y_size = 42     # Number of keys

X_train = None
y_train = None

X_test = None
y_test = None

# Normalize
X = Normalizer().fit_transform(X)

# Standardize
X = StandardScaler().fit_transform(X)

# PCA
#pca = decomposition.PCA(n_components=3)
#pca.fit(X)
#X = pca.transform(X)

In [59]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

K = 5

# Create and fit model
neigh = KNeighborsClassifier(n_neighbors=K)
neigh.fit(X, y)

# Test
pics_nokey, info = get_pics_from_file("Hackaton/data/pics_NOKEY.bin")
print(pics_nokey[0].reshape(1, -1).shape)
print(neigh.predict(pics_nokey[5].reshape(1, -1)))

Ouverture du fichier de pics Hackaton/data/pics_NOKEY.bin
(1, 17)
['pics_NOKEY']


In [ ]:
# NN
import tensorflow as tf

# Ceate model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(X_size, activation='relu'),
    tf.keras.layers.Dense(y_size)
])

# Build model
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Debug
model.summary()

# Train
batch_size = 128
epochs = 15

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

# Test
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])